# Create Plot

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation
from aiida.orm.data.parameter import ParameterData

import ipywidgets as ipw
from IPython.display import display, clear_output, Image

%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
def search():
    """Query AiiDA database"""
    
    query_message.value = "searching..."
    
    filters = {}
    pk_list = inp_pks.value.strip().split()
    if pk_list:
        filters['id'] = {'in': pk_list}
            
    def add_range_filter(bounds, label):
        filters['attributes.'+label] = {'and':[{'>=':bounds[0]}, {'<':bounds[1]}]}

    add_range_filter(inp_dens.value, "density")
    add_range_filter(inp_del_cap.value, "deliverable_capacity")
    add_range_filter(inp_abs_uptake_hp.value, "absolute_methane_uptake_high_P")
    add_range_filter(inp_abs_uptake_lp.value, "absolute_methane_uptake_low_P")

    
    qb = QueryBuilder()        
    qb.append(ParameterData,
          filters=filters,
          project =
           [
               'attributes.'+inp_x.value,
               'attributes.'+inp_y.value,
           ]
         )

    nresults = qb.count()
    if nresults == 0:
        query_message.value = "No results found."
        return

    query_message.value = "{} results found. Plotting...".format(nresults)
    
    l = qb.all()
    res = map(list, zip(*l))
    x = map(float, res[0])
    y = map(float, res[1])


    title = "{} vs {}".format(inp_y.label, inp_x.label)
    
    qb = QueryBuilder()
    qb.append(ParameterData,
          filters=filters,
          project =
           [
               'attributes.'+inp_x.value+'_units',
               'attributes.'+inp_y.value+'_units',
           ]
         )
    qb.limit(1)
    unit_x = ' ['+qb.all()[0][0]+']'
    unit_y = ' ['+qb.all()[0][1]+']'
    
    
    plt_mode = 'plotly'
    if plt_mode is 'plotly':
        plot_plotly(x,y,title=title, xtitle=inp_x.label+unit_x, ytitle=inp_y.label+unit_y)
    else:
        plt.plot(x, y, 'bo', title=title)
    
    query_message.value = "Plotted {} results.".format(nresults)
    
def plot_plotly(x, y, title=None, xtitle=None, ytitle=None):
    import plotly.offline as py
    import plotly.graph_objs as go
    py.init_notebook_mode(connected=True)
    
    # to be changed to rest url for Rocio's DB
    rest_url = 'https://www.materialscloud.org/explore/sssp/details'
    # to be read from QB
    uuid = '81711b8c-1d96-432b-9fd6-813115f7f2e5'
    
    trace = go.Scatter(x=x, y=y, mode='markers', 
                       marker=dict(size=10, line=dict(width=2)))
    
    N = len(x)
    # workaround for links - for proper handling of click events use plot.ly dash
    links = [ dict(x=x[i], y=y[i], text='<a href="{}/{}">o</a>'.format(rest_url, uuid),
                   showarrow=False, font=dict(color='#ffffff'), opacity=0.1) for i in range(N)]
    
    
    if w_links.value:
        layout = go.Layout(annotations=links, title=title, xaxis=dict(title=xtitle), yaxis=dict(title=ytitle))
    else:
        layout = go.Layout(title=title, xaxis=dict(title=xtitle), yaxis=dict(title=ytitle))
    fig = go.Figure(data=[trace], layout=layout)
    py.iplot(fig, filename='jupyter/basic-scatter')

    

In [ ]:
# search UI
style = {"description_width":"220px"}
layout = ipw.Layout(width="90%")
inp_pks = ipw.Text(description='PKs', placeholder='e.g. 1006 1009 (space separated)', layout=layout, style=style)

def slider(desc, min, max):
    return ipw.FloatRangeSlider(description=desc, min=min, max=max, 
                                    value=[min, max], step=0.05, layout=layout, style=style)

inp_dens = slider("Density [kg/m^3]:", 10.0, +2000.0)
inp_del_cap = slider("Del. cap. [v STP/v]:", 0.0, 300.0)
inp_abs_uptake_hp = slider("Abs. meth. uptake high P [mol/kg]:", 0.0, 200.0)
inp_abs_uptake_lp = slider("Abs meth. uptake low P [mol/kg]:",  0.0, 200.0)
search_crit = [inp_pks, inp_dens, inp_del_cap, inp_abs_uptake_hp, inp_abs_uptake_lp]

In [ ]:
def on_click(b):
    with info_out:
        clear_output()
        search()

button = ipw.Button(description="Plot")
button.on_click(on_click)

plot_options = {
    'Density': 'density', 
    'Deliverable capacity': 'deliverable_capacity', 
    'Methane uptake High-P': 'absolute_methane_uptake_high_P', 
    'Methane uptake Low-P': 'absolute_methane_uptake_low_P'
}

inp_x = ipw.Dropdown(
    options = plot_options,
    value = 'density',
    description='X:',
)

inp_y = ipw.Dropdown(
    options = plot_options,
    value='deliverable_capacity',
    description='Y:',
)



w_links = ipw.Checkbox(
    value=False,
    description='With links (slow)',
)



properties = ipw.HBox([inp_x, inp_y])

results = ipw.HTML()
info_out = ipw.Output()
query_message = ipw.HTML("press Plot")
app = ipw.VBox(children=search_crit + [properties, ipw.HBox([button, w_links]), query_message, results, info_out])
display(app)